In [2]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import Video, HTML

1. Read Video Sequence

In [36]:
video_file = "DATA/STGEORGES.avi"

Since the browser can't read avi files, we'll read the video (STGEORGES) using opencv's function `VideoCapture` and show it frame by frame. It's necessary to know the framerate of the video in order to include the appropriate pause between each frame.

In [38]:
# Read the video from the file
source = cv2.VideoCapture(video_file)

framerate = source.get(5)

print("Framerate:", framerate)

Framerate 8.0


Since the frame rate is 8, it's important to include a 125ms pause between each frame

In [39]:
while(source.isOpened()):
    ret, frame = source.read()
    if ret==True:
        cv2.imshow('Frame',frame)
        cv2.waitKey(125)
    else:
        break

source.release()
cv2.destroyAllWindows()

2. Transform to grayscale

To transform the video to grayscale, we follow the same process as the previous but before showing each frame, we convert it to grayscale and save it to a new file. We use `VideoWriter_fourcc` for the video format and `VideoWriter` to write the new video

In [76]:
source = cv2.VideoCapture(video_file)

fourcc = cv2.VideoWriter_fourcc(*'MJPG')
grayscale = cv2.VideoWriter('gray.avi',fourcc, 8., (352,288))

while(source.isOpened()):
    ret, frame = source.read()
    if ret==True:
        # convert the current frame to grayscale
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # reconvert it to BGR to have three channels (the image will still not have colors)
        frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
        
        # write to the new file
        grayscale.write(frame.astype('uint8'))
        cv2.imshow('Frame',frame)
        cv2.waitKey(125)
    else:
        break

source.release()
cv2.destroyAllWindows()

3. Reduce framerate

In [4]:
source = cv2.VideoCapture("gray.avi")

low_fr = cv2.VideoWriter('gray_lower_framerate.avi',
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         4., (352,288))

frames = []
while(source.isOpened()):
    ret, frame = source.read()
    if ret==True:
        
        frames.append(frame)
        # write to the new file
        low_fr.write(frame.astype('uint8'))
    else:
        break

source.release()

4. Video sequence from PIETON images

In [12]:
pieton = cv2.VideoWriter(
    'pieton.avi',
    cv2.VideoWriter_fourcc(*'MJPG'),
    8.,
    (320,275))

# we have 16 images
for i in range(1, 16):
    # reach each frame
    frame = cv2.imread("DATA/PIETON{}.bmp".format(i))

    try:
        # Write the output video 
        pieton.write(frame.astype('uint8'))
        # Display the resulting frame
        cv2.imshow('frame',frame)
    except:
        print("Error processing the frame")

pieton.release()
cv2.destroyAllWindows()

5. Object tracking on the PIETON video

For the object tracking, we will use the HOG descriptor for people detection and use the bounding box coordinates to trace the objects movement.

There are two people on the video, before the detection starts, the user must choose either left or right by clicking the left or right arrow in order to detect the person on the left or the right.

An example of the tracking of the person on the right is on the file pieton_track_right.avi 

In [13]:
# initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

cv2.startWindowThread()

out = cv2.VideoWriter(
    'pieton_track_right.avi',
    cv2.VideoWriter_fourcc(*'MJPG'),
    8.,
    (320,275))

cv2.imshow("image",cv2.imread("DATA/PIETON1.bmp"))
key = cv2.waitKey()

pts = np.array([] ,np.int32)
color = (255, 0, 0)
thickness = 2

for i in range(1, 16):
    frame = cv2.imread("DATA/PIETON{}.bmp".format(i))

    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

    # detect people in the image
    # returns the bounding boxes for the detected objects
    boxes, weights = hog.detectMultiScale(frame, winStride=(4,4) )
    
    boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in boxes])
    
    try:

        if(key == 83):
            boxes_select = max(box[0] for box in boxes)
        elif(key == 81):
            boxes_select = min(box[0] for box in boxes)
        else:
            boxes_select = max(box[0] for box in boxes)
            
        boxes = [box for box in boxes if box[0] == boxes_select]

        for (xA, yA, xB, yB) in boxes:
            # display the detected boxes 
            cv2.rectangle(frame, (xA, yA), (xB, yB),
                              (0, 255, 0), 2)
            point = [int((xA+xB)/2), int((yA+yB)/2)]
            pts = np.append(pts, point)
            pts = pts.reshape((-1, 1, 2))
            cv2.polylines(frame, [pts],
                      False, color, thickness)
        # Write the output video 
        out.write(frame.astype('uint8'))
        # Display the resulting frame
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    except:
        print("exception")

out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

exception
exception


-1